In [4]:
# Importing the necessary libraries that will be used.

import json
import requests
import pandas as pd
import numpy as np
import random
import sqlalchemy

In [2]:
# Creating an empty list to append the jsons of all the videos in it.

df = []

In [3]:
# Opening the txt file that I save the list in, so I don't have to access the API again for the same video id.

with open('Redtube_list.txt', 'r') as filehandle:
    df = json.load(filehandle)

In [2]:
"""
I used random selection to go through the API, because I couldn't access all the videos on the website at once, 
I had to access each video individually using its id, I could only access the API 30000 times per day,
in each time using different video id, some of the video IDs were empty, this is why out of 100000 time to 
access the API over 3 days, only 51900 videos were accessed.
I had to run the code over several days to get the needed information,
this is why I was saving the list to the PC so I can open it later and append the new data.
""""


for i in random.sample(range(0, 1500000), 100000):
    url = f'https://api.redtube.com/?data=redtube.Videos.getVideoById&video_id={i}&output=json&thumbsize=all'
    response = requests.get(url)
    print(response.status_code)
    if response.status_code == 200:
        data = response.json()
        df.append(data)
        print(f'Finished df temp for number {i}')   
print('Completed')

In [8]:
# Unpacking the dictionaries that contain the information about the video from the list.

video = []
for i in df:
    for key, value in i.items():
        if key == 'video':
            video.append(value)

In [9]:
# Unpacking the dictionaries that contain the information about the tage/categories of each video from the list.

tags = []
for l in video:
    for k,v in l.items():
        if k == 'tags':
            tags.append(v)

In [10]:
# Creating empty arrays to append the needed information in each array

views = np.array([])
rating = np.array([])
titles = np.array([])

for dict1 in video:
    for key1, value1 in dict1.items():
        if key1 == 'views':
            views = np.append(views, np.array([value1]))
        elif key1 == 'rating':
            rating = np.append(rating, np.array([value1]))
        elif key1 == 'title':
            titles = np.append(titles, np.array([value1]))          

In [11]:
# Creating a table from the list containing tags/categories.

categories = pd.DataFrame(tags)

In [12]:
# Dropping the unnecessary categories as we agreed on only including the first three categories for each video.

categories = categories.drop(categories.columns[[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]], axis=1)

In [13]:
# Renaming the columns because they had numbers assigned to each column

categories.columns = ['Category_1', 'Category_2', 'Category_3']

In [14]:
# Combining the three arrays containing information about the (title of video, number of views & ratings) in one table.

arrays = {'Title': titles, 'Number_of_Views': views, 'Rating': rating}
table = pd.DataFrame(data=arrays)

In [15]:
# Merging the table with the videos information with the table containing the categories.

full_table = pd.merge(table, categories, left_index=True, right_index=True)

In [16]:
# Adding a new column with the website name, to be able to distinguish the data, because we had three different sources.

full_table['Website'] = 'Redtube'

In [17]:
# Changing the type of data for each column with the type that we had agreed upon.

full_table = full_table.astype({'Title': str, 'Number_of_Views': int, 'Rating': float, 'Category_1': str, 'Category_2': str, 'Category_3': str, 'Website': str})

In [18]:
#Making sure that the data types were changed accordingly. 

full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51901 entries, 0 to 51900
Data columns (total 7 columns):
Title              51901 non-null object
Number_of_Views    51901 non-null int32
Rating             51901 non-null float64
Category_1         51901 non-null object
Category_2         51901 non-null object
Category_3         51901 non-null object
Website            51901 non-null object
dtypes: float64(1), int32(1), object(5)
memory usage: 2.6+ MB


In [19]:
# Checking the head of the table, to make sure that it had the same order that we agreed upon.

full_table.head()

,Title,Number_of_Views,Rating,Category_1,Category_2,Category_3,Website
0,Glamour girl pleasuring herself,215864,88.1962,Amateur,Blonde,Caucasian,Redtube
1,Sexy artist seducing and bodypainting,107805,87.3641,Brunette,Glamour,Latin,Redtube
2,Sexy teen webcam striptease,627376,88.1806,Amateur,Black-haired,Latin,Redtube
3,Sweet girl deepthroating,152832,90.7239,Bikini,Blowjob,Caucasian,Redtube
4,Cybersex mistake,117170,88.1584,Caucasian,Couple,Funny,Redtube


In [20]:
#Saving the table to PC as a form of BU.

full_table.to_csv(r'C:\\Users\\user\\Downloads\\Data Analytics\\Directory\\Project-Week-3-Data-Thieves\\your-project\\Redtube_table.csv')

In [21]:
# Establishing connection with the online shared database.

driver = 'mysql+pymysql'
user = 'root'
password = 'Ironhack10'
ip = '35.204.21.134'
database = 'porn_analysis'

conn_string = f'{driver}://{user}:{password}@{ip}/{database}'
    
conn = sqlalchemy.create_engine(conn_string)

In [34]:
# Pushing the table to the online shared database.

full_table.to_sql('main', conn, if_exists = 'append')

In [49]:
# Used this cell while getting the data from the API, to save the list and append the new data to the same list.

with open('Ali_list.txt', 'w') as filehandle:
    json.dump(df, filehandle)